# 🤖 Chatbot_v2.ipynb

## Description

This notebook implements an improved multilingual chatbot using `facebook/blenderbot_small-90M` for dialogue and `persiannlp/mt5-small-parsinlu` models for translation. Improvements include better context handling, cleaner translations, and more natural responses.

# 1️⃣ Install dependencies

In [ ]:
!pip install transformers torch langdetect

# 2️⃣ Import libraries

In [ ]:
from transformers import BlenderbotSmallForConditionalGeneration, BlenderbotSmallTokenizer
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from langdetect import detect
import torch, re

# 3️⃣ Load BlenderBot model

In [ ]:
blender_model_name = 'facebook/blenderbot_small-90M'
blender_tokenizer = BlenderbotSmallTokenizer.from_pretrained(blender_model_name)
blender_model = BlenderbotSmallForConditionalGeneration.from_pretrained(blender_model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
blender_model.to(device)

# 4️⃣ Load translation models

In [ ]:
fa_en_model_name = 'persiannlp/mt5-small-parsinlu-opus-translation_fa_en'
en_fa_model_name = 'persiannlp/mt5-small-parsinlu-translation_en_fa'

fa_en_tokenizer = MT5Tokenizer.from_pretrained(fa_en_model_name, legacy=False)
fa_en_model = MT5ForConditionalGeneration.from_pretrained(fa_en_model_name)
en_fa_tokenizer = MT5Tokenizer.from_pretrained(en_fa_model_name, legacy=False)
en_fa_model = MT5ForConditionalGeneration.from_pretrained(en_fa_model_name)

fa_en_model.to(device)
en_fa_model.to(device)

# 5️⃣ Helper functions

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'en'

def clean_translation(text):
    text = re.sub(r'کاربر[:：]\s*', '', text)
    text = re.sub(r'Bot[:：]\s*', '', text)
    text = text.strip()
    return text

def translate_text(text, source_lang, target_lang):
    if source_lang == target_lang:
        return text

    if source_lang == 'fa' and target_lang == 'en':
        tokenizer, model = fa_en_tokenizer, fa_en_model
    else:
        tokenizer, model = en_fa_tokenizer, en_fa_model

    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clean_translation(translated)

def generate_response(user_input, conversation_history):
    input_lang = detect_language(user_input)
    
    if input_lang == 'fa':
        input_text_en = translate_text(user_input, 'fa', 'en')
    else:
        input_text_en = user_input

    conversation_history.append(f'User: {input_text_en}')
    context = ' '.join(conversation_history[-4:])  # use last 4 messages for context

    inputs = blender_tokenizer(context, return_tensors='pt', truncation=True, max_length=512).to(device)
    reply_ids = blender_model.generate(
        **inputs,
        max_length=120,
        num_beams=5,
        no_repeat_ngram_size=3,
        length_penalty=1.1,
        top_p=0.9,
        temperature=0.85,
        early_stopping=True
    )
    response_en = blender_tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    response_en = re.sub(r'User[:：]\s*', '', response_en).strip()

    if input_lang == 'fa':
        response = translate_text(response_en, 'en', 'fa')
    else:
        response = response_en

    conversation_history.append(f'Bot: {response_en}')
    return response, conversation_history

# 6️⃣ Test the improved chatbot

In [ ]:
conversation_history = []

print('🟢 English Conversation Test:\n')
user_input = 'Hello! How are you today?'
response, conversation_history = generate_response(user_input, conversation_history)
print(f'User: {user_input}\nBot: {response}\n')

user_input = 'What do you like to do for fun?'
response, conversation_history = generate_response(user_input, conversation_history)
print(f'User: {user_input}\nBot: {response}\n')

print('🟣 Persian Conversation Test:\n')
user_input = 'سلام! امروز چطور هستی؟'
response, conversation_history = generate_response(user_input, conversation_history)
print(f'کاربر: {user_input}\nربات: {response}\n')

user_input = 'برای سرگرمی چه کارهایی انجام می‌دهی؟'
response, conversation_history = generate_response(user_input, conversation_history)
print(f'کاربر: {user_input}\nربات: {response}\n')